# 映画レビューデータの傾向分析

本ノートブックでは、独自に生成したダミーのレビューデータ（`dummy_reviews.csv`）を用いて、映画レビューの傾向を分析します。

## 0. 準備：ライブラリのインストール
日本語表示と形態素解析のために必要なライブラリをインストールします。

In [ ]:
!pip install japanize-matplotlib janome

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib
from janome.tokenizer import Tokenizer
from collections import Counter

# グラフのスタイル設定
sns.set(font="IPAexGothic") # または japanize_matplotlib が自動で設定します

## 1. データの読み込みとクリーニング

In [ ]:
df = pd.read_csv('dummy_reviews.csv')

# 星評価の数値化と欠損値処理
df['星評価'] = pd.to_numeric(df['星評価'], errors='coerce')
df = df.dropna(subset=['星評価'])

# 日付の変換
df['投稿日'] = pd.to_datetime(df['投稿日'])

print(f"有効なデータ件数: {len(df)}件")
df.head()

## 2. 統計的な可視化（星評価の分布）

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(df['星評価'], bins=10, kde=True, color='skyblue')
plt.title('映画レビューの星評価分布')
plt.xlabel('評価（星の数）')
plt.ylabel('レビュー数')
plt.show()

## 3. 時系列分析（投稿数の推移）

In [ ]:
df['月'] = df['投稿日'].dt.to_period('M')
monthly_counts = df.groupby('月').size()

plt.figure(figsize=(12, 5))
monthly_counts.plot(kind='line', marker='o', color='green')
plt.title('月別レビュー投稿数の推移')
plt.xlabel('年月')
plt.ylabel('投稿数')
plt.grid(True)
plt.show()

## 4. 頻出単語の分析（テキストマイニング）
口コミ全文から名詞を抽出し、頻出語を確認します。

In [ ]:
t = Tokenizer()
words = []

for text in df['口コミ全文']:
    tokens = t.tokenize(text)
    for token in tokens:
        # 名詞のみを抽出（1文字の単語や一般的な助詞を除外するなどの処理が本来は必要）
        part_of_speech = token.part_of_speech.split(',')[0]
        if part_of_speech == '名詞' and len(token.surface) > 1:
            words.append(token.surface)

# 頻出上位10語を取得
common_words = Counter(words).most_common(10)
word_df = pd.DataFrame(common_words, columns=['単語', '出現回数'])

plt.figure(figsize=(10, 5))
sns.barplot(x='出現回数', y='単語', data=word_df, palette='viridis')
plt.title('口コミ中の頻出単語ランキング（名詞）')
plt.xlabel('出現回数')
plt.ylabel('単語')
plt.show()

## 5. 分析結果の考察

- **評価の傾向**: 星評価の分布から、作品の全体的な満足度を把握しました。
- **トレンドの把握**: 投稿数の推移から、特定の時期に反響が大きかったことが確認できました。
- **キーワード分析**: 頻出語から、「映像」「ストーリー」「演技」といった要素が注目されていることが分かりました。